In [6]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [8]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()
pass_id = test_data.PassengerId

In [ ]:
train_data.describe()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
from scipy.stats.stats import pearsonr

print("Correlations")
age_data = pd.get_dummies(train_data[["Survived", "Age"]])
age_mean = age_data['Age'].mean()
age_data_drop = age_data.dropna()
corr = pearsonr(age_data_drop["Survived"], age_data_drop["Age"])
print("Raw age: ", corr)

age_data['Age'].fillna(value=age_mean, inplace=True)
corr = pearsonr(age_data["Survived"], age_data["Age"])
print("Filled age: ", corr)

correlated_columns = ["Pclass", "SibSp", "Parch", "Fare"]
for name in correlated_columns:
    corr = pearsonr(train_data['Survived'], train_data[name])
    print(name, ": ", corr)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.ensemble import AdaBoostClassifier
import statistics as st

features = ["Pclass", "Sex", "Fare", "Parch", "Age"]
age_mean = train_data['Age'].mean()
train_data["Age"].fillna(value=age_mean, inplace=True)
X_all = pd.get_dummies(train_data)
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]

model_names = ["forest", "SVC", "logic"]
forest_train = []
forest_test = []
sgd_train = []
sgd_test = []
svc_train = []
svc_test = []
logic_train = []
logic_test = []

for i in range (10):
    X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.8)
    
    pipe = Pipeline([("scaler", StandardScaler()),
      ("model", RandomForestClassifier(n_estimators=100, max_depth=8))])
    pipe.fit(X_train, y_train)
    forest_train.append(f1_score(pipe.predict(X_train), y_train))
    forest_test.append(f1_score(pipe.predict(X_test), y_test))

    pipe = Pipeline([("scaler", StandardScaler()),
      ("model", SVC())])
    pipe.fit(X_train, y_train)
    svc_train.append(f1_score(pipe.predict(X_train), y_train))
    svc_test.append(f1_score(pipe.predict(X_test), y_test))

    pipe = Pipeline([("scaler", StandardScaler()),
      ("model", LogisticRegression(solver='lbfgs', max_iter=200))])
    pipe.fit(X_train, y_train)
    logic_train.append(f1_score(pipe.predict(X_train), y_train))
    logic_test.append(f1_score(pipe.predict(X_test), y_test))

for name in model_names:
    if name == "forest":
        train = forest_train
        test = forest_test
    elif name == "SVC":
        train = svc_train
        test = svc_test
    elif name == "logic":
        train = logic_train
        test = logic_test
        
    print(name)
    print("training score")
    print("Min: ", min(train))
    print("Max: ", max(train))
    print("Mean: ", st.mean(train))
    print("testing score")
    print("Min: ", min(test))
    print("Max: ", max(test))
    print("Mean: ", st.mean(test))
    print("")

In [ ]:
# Ensamble time!
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier

fold = model_selection.KFold(n_splits=5, shuffle=True)
estimators = []
estimators.append(("forest", 
          RandomForestClassifier(n_estimators=100, max_depth=8)))
estimators.append(("svc", SVC()))
estimators.append(("logic", 
                  LogisticRegression(solver='lbfgs', max_iter=150)))
ensamble = VotingClassifier(estimators)
fin = model_selection.cross_val_score(ensemble, X, y, cv=fold)
print(fin.mean())
ensamble.fit(X_train,y_train)
print(f1_score(ensamble.predict(X_test), y_test))
ensamble.fit(X,y)
print(f1_score(ensamble.predict(X_test), y_test))

fare_mean = test_data['Fare'].mean()
test_data["Fare"].fillna(value=fare_mean, inplace=True)
age_mean_test = test_data['Age'].mean()
test_data["Age"].fillna(value=age_mean_test, inplace=True)
test_data = pd.get_dummies(test_data[features])

predictions = ensamble.predict(test_data)
output = pd.DataFrame({'PassengerId': pass_id, 'Survived': predictions})
output.to_csv('submission.csv', index=False)